In [163]:
from tvDatafeed import TvDatafeed, Interval
import mplfinance as mpf
import pandas as pd
import ta  # module for technical indicators

In [164]:
symbol = "AMD"
exchange = "NASDAQ"
interval = Interval.in_1_minute 
n_bars = 5000
tv = TvDatafeed()

In [165]:
df = tv.get_hist(symbol=symbol, exchange=exchange, interval=interval, n_bars=n_bars)

In [166]:
df.info

<bound method DataFrame.info of                          symbol    open    high      low   close   volume
datetime                                                                 
2024-12-19 20:10:00  NASDAQ:AMD  119.50  119.50  119.380  119.39   9506.0
2024-12-19 20:11:00  NASDAQ:AMD  119.37  119.48  119.360  119.44   2831.0
2024-12-19 20:12:00  NASDAQ:AMD  119.45  119.45  119.320  119.34   7082.0
2024-12-19 20:13:00  NASDAQ:AMD  119.34  119.35  119.210  119.30  12842.0
2024-12-19 20:14:00  NASDAQ:AMD  119.31  119.34  119.260  119.34   4998.0
...                         ...     ...     ...      ...     ...      ...
2025-01-10 21:55:00  NASDAQ:AMD  116.00  116.23  115.970  116.12  33440.0
2025-01-10 21:56:00  NASDAQ:AMD  116.11  116.16  116.030  116.08  23677.0
2025-01-10 21:57:00  NASDAQ:AMD  116.09  116.16  116.080  116.11  29499.0
2025-01-10 21:58:00  NASDAQ:AMD  116.10  116.16  116.090  116.13  17173.0
2025-01-10 21:59:00  NASDAQ:AMD  116.13  116.17  115.965  115.98  66140.0

[5000

We will create a dataset using data from TradingView. The target label will be a boolean indicating whether the stock price increased (1) or decreased (0). The dataset will include features like RSI (Relative Strength Index) and other relevant indicators to give the model a comprehensive view of the stock's historical behavior.

In [167]:
n = len(df)
m = 10 # how many previous bars we want to include

df = df.drop(columns=['symbol'])

In [168]:
# adding EMA (Exponential Moving Average)
df['EMA'] = df['close'].ewm(span=m).mean()

In [169]:
from ta.momentum import RSIIndicator

# adding RSI (Relative Strenght Index)
df['RSI'] = ta.momentum.RSIIndicator(close=df['close'], window=m).rsi()

In [170]:
import ta.trend

# adding MACD (Moving Average Convergence Divergence)
macd = ta.trend.MACD(close=df['close'])
df['MACD'] = macd.macd()
df['MACD_Signal'] = macd.macd_signal()

In [171]:
# % change of 2 concecutive stock prices
df["%_Change"] = df['close'].pct_change()

In [172]:
# volume % change 
df['Volume_Change'] = df['volume'].pct_change()

In [173]:
import ta.volatility

# adding ATR (Average True Range)
df['ATR'] = ta.volatility.AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=m).average_true_range()

In [174]:
bb = ta.volatility.BollingerBands(close=df['close'], window=m, window_dev=2)
df['BB_High'] = bb.bollinger_hband()
df['BB_Low'] = bb.bollinger_lband()

In [175]:
# adding OHLCV for last m bars
for i in range(1, m+1):
    df[f'close_t-{i}'] = df['close'].shift(i)
    df[f'volume_t-{i}'] = df['volume'].shift(i)
    df[f'RSI_t-{i}'] = RSIIndicator(close=df['close'].shift(i), window=14).rsi()
    df[f"%_Change_t-{i}"] = df['close'].shift(i).pct_change()       # after adding this and volume % accuracy dropped
    df[f'Volume_Change_t-{i}'] = df['volume'].shift(i).pct_change()

In [176]:
# Adding Target Column
df['Target'] = (df['close'].shift(-3) > df['close']).astype(int)

In [177]:
# removing incomplete rows
df = df.dropna()

In [178]:
df.head(5)

,open,high,low,close,volume,EMA,RSI,MACD,MACD_Signal,%_Change,Volume_Change,ATR,BB_High,BB_Low,close_t-1,volume_t-1,RSI_t-1,%_Change_t-1,Volume_Change_t-1,close_t-2,volume_t-2,RSI_t-2,%_Change_t-2,Volume_Change_t-2,close_t-3,volume_t-3,RSI_t-3,%_Change_t-3,Volume_Change_t-3,close_t-4,volume_t-4,RSI_t-4,%_Change_t-4,Volume_Change_t-4,close_t-5,volume_t-5,RSI_t-5,%_Change_t-5,Volume_Change_t-5,close_t-6,volume_t-6,RSI_t-6,%_Change_t-6,Volume_Change_t-6,close_t-7,volume_t-7,RSI_t-7,%_Change_t-7,Volume_Change_t-7,close_t-8,volume_t-8,RSI_t-8,%_Change_t-8,Volume_Change_t-8,close_t-9,volume_t-9,RSI_t-9,%_Change_t-9,Volume_Change_t-9,close_t-10,volume_t-10,RSI_t-10,%_Change_t-10,Volume_Change_t-10,Target
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-12-19 20:43:00,118.84,118.930,118.81,118.85,7965.0,119.030649,25.217323,-0.105406,-0.067127,0.000252,-0.634868,0.125375,119.323149,118.792851,118.82,21814.0,25.503735,-0.000841,-0.307140,118.92,31484.0,29.062797,-0.001344,-0.539101,119.08,68310.0,36.664543,-0.000587,10.793854,119.15,5792.0,41.023718,-0.000168,0.640329,119.17,3531.0,42.359883,0.000504,-0.829354,119.11,20692.0,36.608203,-0.000168,2.975408,119.13,5205.0,37.774926,-0.000336,-0.333803,119.17,7813.0,40.151412,-0.000084,-0.054460,119.18,8263.0,40.746494,-0.000671,-0.176089,119.26,10029.0,45.787648,-0.000419,1.401580,1
2024-12-19 20:44:00,118.86,118.940,118.85,118.92,6685.0,119.010513,33.712692,-0.108515,-0.075404,0.000589,-0.160703,0.121838,119.295181,118.768819,118.85,7965.0,28.338958,0.000252,-0.634868,118.82,21814.0,25.503735,-0.000841,-0.307140,118.92,31484.0,29.062797,-0.001344,-0.539101,119.08,68310.0,36.664543,-0.000587,10.793854,119.15,5792.0,41.023718,-0.000168,0.640329,119.17,3531.0,42.359883,0.000504,-0.829354,119.11,20692.0,36.608203,-0.000168,2.975408,119.13,5205.0,37.774926,-0.000336,-0.333803,119.17,7813.0,40.151412,-0.000084,-0.054460,119.18,8263.0,40.746494,-0.000671,-0.176089,1
2024-12-19 20:45:00,118.90,118.960,118.89,118.96,11258.0,119.001322,38.172176,-0.106523,-0.081628,0.000336,0.684069,0.116654,119.259910,118.762090,118.92,6685.0,34.594020,0.000589,-0.160703,118.85,7965.0,28.338958,0.000252,-0.634868,118.82,21814.0,25.503735,-0.000841,-0.307140,118.92,31484.0,29.062797,-0.001344,-0.539101,119.08,68310.0,36.664543,-0.000587,10.793854,119.15,5792.0,41.023718,-0.000168,0.640329,119.17,3531.0,42.359883,0.000504,-0.829354,119.11,20692.0,36.608203,-0.000168,2.975408,119.13,5205.0,37.774926,-0.000336,-0.333803,119.17,7813.0,40.151412,-0.000084,-0.054460,1
2024-12-19 20:46:00,118.96,119.090,118.87,119.06,12015.0,119.011997,47.907058,-0.095772,-0.084457,0.000841,0.067241,0.126989,119.242864,118.765136,118.96,11258.0,37.928203,0.000336,0.684069,118.92,6685.0,34.594020,0.000589,-0.160703,118.85,7965.0,28.338958,0.000252,-0.634868,118.82,21814.0,25.503735,-0.000841,-0.307140,118.92,31484.0,29.062797,-0.001344,-0.539101,119.08,68310.0,36.664543,-0.000587,10.793854,119.15,5792.0,41.023718,-0.000168,0.640329,119.17,3531.0,42.359883,0.000504,-0.829354,119.11,20692.0,36.608203,-0.000168,2.975408,119.13,5205.0,37.774926,-0.000336,-0.333803,1
2024-12-19 20:47:00,119.06,119.205,119.02,119.18,8686.0,119.042558,56.945679,-0.076684,-0.082902,0.001008,-0.277070,0.132790,119.265472,118.756528,119.06,12015.0,45.419154,0.000841,0.067241,118.96,11258.0,37.928203,0.000336,0.684069,118.92,6685.0,34.594020,0.000589,-0.160703,118.85,7965.0,28.338958,0.000252,-0.634868,118.82,21814.0,25.503735,-0.000841,-0.307140,118.92,31484.0,29.062797,-0.001344,-0.539101,119.08,68310.0,36.664543,-0.000587,10.793854,119.15,5792.0,41.023718,-0.000168,0.640329,119.17,3531.0,42.359883,0.000504,-0.829354,119.11,20692.0,36.608203,-0.000168,2.975408,1


Now our dataset is ready to use. Ofc we dont have to use every feature

In [179]:
df.to_csv('../../data/dataset.csv', index=False)